<span style="color:orange">
    <font size="7"> <strong>Where Should I Live?</strong></font>
</span>
<font size="5"><br/>Programming for Data Science Project 2025 - Group 14</font>


<font size="4"><br/>Project by:
  <ul>
    <li>Constança Rocha - 20241758</li>
    <li>Inês Caseiro - 20241780</li>
    <li>Tiago Silveira - 20241698</li>
  </ul>
</font>

<strong>GitHub Repository:</strong>
<br>
<href>https://github.com/CoderCactus/WhereShouldILive/</href>


This project aims to explore the living conditions of several European cities using the information contained in the ``city_data`` dataset. We will do the basic data preprocessing (handling missing values, duplicates and columns mismatch), analysize the data and extract some insights about the cities' living conditions, create a interactive map by employing web scrapping to obtain the coordinates, and build a recomendation system.

### Library Installation

In [443]:
#!pip install pandas
#!pip install beautifulsoup4  
#!pip install selenium
#!pip install requests
#!pip install panel
#!pip install jupyter_bokeh
#!pip install lat-lon-parser

In [444]:
import pandas as pd
import numpy as np
import datetime as dt


import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)


### File Importing

In [445]:
df = pd.read_csv("city_data.csv", delimiter='|')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00


### Header Correction

In [446]:
new_header = df.iloc[0]

df.columns = new_header
df = df[1:]

df.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00
2,"Salzburg, Austria",243,375489,250472,20.3,3,66689,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00
3,"Brussels, Belgium",681,3284548,2137425,27.5,10.7,62500,3,"French, Dutch, Arabic, English",3350,1200,1900,NaN,2023-04-22 00:00:00
4,"Antwerp, Belgium",928,1139663,723396,27.7,6.2,57595,3,"Dutch, French, Arabic",2609,900,1953,NaN,2024-08-09 00:00:00
5,"Gent, Belgium",552,645813,417832,24.8,NaN,53311,2,"Dutch, French",2400,827,1200,120,2023-07-17 00:00:00


### Column Name Correction

In [447]:
df = df.rename(columns = {'Avgerage Rent Price': 'Average Rent Price'})

## Data Type Correction

In [448]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 1 to 86
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   City                             86 non-null     object
 1   Population Density               85 non-null     object
 2   Population                       86 non-null     object
 3   Working Age Population           85 non-null     object
 4   Youth Dependency Ratio           86 non-null     object
 5   Unemployment Rate                85 non-null     object
 6   GDP per Capita                   85 non-null     object
 7   Days of very strong heat stress  86 non-null     object
 8   Main Spoken Languages            85 non-null     object
 9   Average Monthly Salary           86 non-null     object
 10  Average Rent Price               86 non-null     object
 11  Average Cost of Living           86 non-null     object
 12  Average Price Groceries          6 non

To optimize memory usage, numeric columns were converted to appropriate integer or float types (having in consideration that int does not accept null values). And categorical variables were converted to category (based on the number of unique values in that collumn)

In [449]:
i8min, i8max, i16min, i16max, i32min, i32max, i64min, i64max = np.iinfo(np.int8).min, np.iinfo(np.int8).max, np.iinfo(np.int16).min, np.iinfo(np.int16).max, np.iinfo(np.int32).min, np.iinfo(np.int32).max, np.iinfo(np.int64).min, np.iinfo(np.int64).max
print(f'int8:[{i8min}, {i8max}]')
print(f'int16:[{i16min}, {i16max}]')
print(f'int32:[{i32min}, {i32max}]')
print(f'int64:[{i64min}, {i64max}]')
    
    
f16min, f16max, f32min, f32max, f64min, f64max = np.finfo(np.float16).min, np.finfo(np.float16).max, np.finfo(np.float32).min, np.finfo(np.float32).max, np.finfo(np.float64).min, np.finfo(np.float64).max
#float8 doesn't exist
print(f'float16:[{f16min}, {f16max}]')
print(f'float32:[{f32min}, {f32max}]')
print(f'float64:[{f64min}, {f64max}]')

int8:[-128, 127]
int16:[-32768, 32767]
int32:[-2147483648, 2147483647]
int64:[-9223372036854775808, 9223372036854775807]
float16:[-65504.0, 65504.0]
float32:[-3.4028234663852886e+38, 3.4028234663852886e+38]
float64:[-1.7976931348623157e+308, 1.7976931348623157e+308]


In [450]:
df = df.astype({'City': 'category',
                    'Population Density': 'float16',
                    'Population': 'int32',
                    'Working Age Population ': 'float32',
                    'Youth Dependency Ratio': 'float16',
                    'Unemployment Rate': 'float16',
                    'GDP per Capita': 'float32',
                    'Days of very strong heat stress': 'int8',
                    'Main Spoken Languages': 'object',
                    'Average Monthly Salary': 'int16',
                    'Average Rent Price': 'int16',
                    'Average Cost of Living': 'int16',
                    'Average Price Groceries': 'float16',
                    'Last Data Update': 'datetime64[ns]'})

In [451]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 1 to 86
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   City                             86 non-null     category      
 1   Population Density               85 non-null     float16       
 2   Population                       86 non-null     int32         
 3   Working Age Population           85 non-null     float32       
 4   Youth Dependency Ratio           86 non-null     float16       
 5   Unemployment Rate                85 non-null     float16       
 6   GDP per Capita                   85 non-null     float32       
 7   Days of very strong heat stress  86 non-null     int8          
 8   Main Spoken Languages            85 non-null     object        
 9   Average Monthly Salary           86 non-null     int16         
 10  Average Rent Price               86 non-null     int16         


## Data Wrangling and Analysis

#### 1. How did you handle missing values and duplicate records in the dataset? Justify your approach.


<font size = '4'> <b> Duplicate Records <b>


As can be seen in the following output there are several duplicate records in our dataset.

In [452]:
temp = df.copy()
temp[temp.duplicated(keep=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
16,Berlin. Germany,304.0,5303922,3481212.0,21.296875,4.699219,46548.0,3,"German, Turkish, Arabic, English",3200,1220,2200,NaN,2023-06-29
17,Berlin. Germany,304.0,5303922,3481212.0,21.296875,4.699219,46548.0,3,"German, Turkish, Arabic, English",3200,1220,2200,NaN,2023-06-29
33,"Malaga, Spain",571.0,869096,585608.0,23.500000,17.703125,27694.0,0,"Spanish, English",2200,1312,1400,NaN,2023-11-27
34,"Malaga, Spain",571.0,869096,585608.0,23.500000,17.703125,27694.0,0,"Spanish, English",2200,1312,1400,NaN,2023-11-27


To keep a consistent dataset we will drop the second record of each of the repeated cities (only keeping on occurence of each).

In [453]:
temp.drop_duplicates(inplace=True)
temp[temp.duplicated(keep=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update


As we can see there are no more duplicate records.

In [454]:
df = temp

<font size = '4'> <b> Missing values <b>

First lets check how many null values (empty observations) we have in each of our dataset's columns.

In [455]:
df.isna().sum()


0
City                                0
Population Density                  1
Population                          0
Working Age Population              1
Youth Dependency Ratio              0
Unemployment Rate                   1
GDP per Capita                      1
Days of very strong heat stress     0
Main Spoken Languages               1
Average Monthly Salary              0
Average Rent Price                  0
Average Cost of Living              0
Average Price Groceries            78
Last Data Update                    0
dtype: int64

As we can see, we have missing values in ```Population Density```, ```Working Age Population```, ```Unemployment Rate```, ```GDP per Capita```, ```Main Spoken Languages``` and ```Average Price Groceries```.

As the column ```Average Price Groceries``` has only 6 values (out of 84) and it would not be needed, we opted for eliminate it.

In [456]:
df.drop('Average Price Groceries', axis = 1, inplace = True)

The numeric missing values (```Population Density```, ```Working Age Population```, ```Unemployment Rate``` and ```GDP per Capita```) will be filled using KNN Imputer to fill them.

In [457]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

numeric_columns = df.select_dtypes(include = 'number').columns
numeric_data = df[numeric_columns]
non_numeric = df.drop(numeric_columns, axis = 1)

original_index = df.index

#Scale the data
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(numeric_data)

#Imputing missing values
imputer = KNNImputer(n_neighbors = 5, weights = 'distance')
numeric_imputed_scaled = imputer.fit_transform(numeric_scaled)

#Reverse the scaling back to the original values
numeric_imputed = scaler.inverse_transform(numeric_imputed_scaled) #array

numeric_df = pd.DataFrame(numeric_imputed, columns = numeric_columns, index = original_index)

#Join the numeric and non-numeric datasets
temp = pd.concat([non_numeric['City'],
                     numeric_df.iloc[:,:7],
                     non_numeric['Main Spoken Languages'],
                     numeric_df.iloc[:,7:],
                     non_numeric['Last Data Update']], axis = 1)
df = temp

The missing value in ````Main Spoken Languages```` is in Dusseldorf, a German city. We conducted some research online and found that this city is among the top German cities where English is spoken. From the cities in our dataset, those English-speaking cities are marked as such, so we decided to fill the missing value with both languages (German and English).

In [458]:
df[df['Main Spoken Languages'].isna()]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update
25,"Dusseldorf, Germany",1295.0,1554077.0,989582.940088,21.40625,3.300781,71238.0,3.0,NaN,3200.0,1100.0,2050.0,2024-01-15


In [459]:
df.fillna({'Main Spoken Languages':'German, English'}, inplace = True)

Now as we can see, all collumns have no missing values

In [460]:
df.isna().sum()

City                               0
Population Density                 0
Population                         0
Working Age Population             0
Youth Dependency Ratio             0
Unemployment Rate                  0
GDP per Capita                     0
Days of very strong heat stress    0
Main Spoken Languages              0
Average Monthly Salary             0
Average Rent Price                 0
Average Cost of Living             0
Last Data Update                   0
dtype: int64

As we can see, while filling in the missing values our columns data types were changed so we need to correct them to the least memory usage possible.

In [461]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 1 to 86
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   City                             84 non-null     category      
 1   Population Density               84 non-null     float64       
 2   Population                       84 non-null     float64       
 3   Working Age Population           84 non-null     float64       
 4   Youth Dependency Ratio           84 non-null     float64       
 5   Unemployment Rate                84 non-null     float64       
 6   GDP per Capita                   84 non-null     float64       
 7   Days of very strong heat stress  84 non-null     float64       
 8   Main Spoken Languages            84 non-null     object        
 9   Average Monthly Salary           84 non-null     float64       
 10  Average Rent Price               84 non-null     float64       
 11  

In [462]:
df = df.astype({'City': 'object',
                'Population Density': 'int16',
                'Population': 'int32',
                'Working Age Population ': 'int32',
                'Youth Dependency Ratio': 'float16',
                'Unemployment Rate': 'float16',
                'GDP per Capita': 'float32',
                'Days of very strong heat stress': 'int8',
                'Main Spoken Languages': 'object',
                'Average Monthly Salary': 'int16',
                'Average Rent Price': 'int16',
                'Average Cost of Living': 'int16',
                'Last Data Update': 'datetime64[ns]'
                })

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, 1 to 86
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   City                             84 non-null     object        
 1   Population Density               84 non-null     int16         
 2   Population                       84 non-null     int32         
 3   Working Age Population           84 non-null     int32         
 4   Youth Dependency Ratio           84 non-null     float16       
 5   Unemployment Rate                84 non-null     float16       
 6   GDP per Capita                   84 non-null     float32       
 7   Days of very strong heat stress  84 non-null     int8          
 8   Main Spoken Languages            84 non-null     object        
 9   Average Monthly Salary           84 non-null     int16         
 10  Average Rent Price               84 non-null     int16         
 11  

#### 2.

 ##### ``a) Which country appears most frequently in the dataset? How many cities are associated with it?``

First we need to separate the city name from the country to which it belongs

The problem is that not all pairs in the City collumn have the same formatting (``city, country``).

In [463]:
temp= df.copy()

pattern = r'\s{2,}|[.;]|^[^,]+$'

# Flag invalid rows
temp[temp['City'].str.contains(pattern, regex=True, na=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update
13,Lemesos;Cyprus,190,262238,173906,23.062500,11.203125,46723.929688,0,"Greek, English, Russian",2500,1400,2300,2023-08-14
16,Berlin. Germany,304,5303922,3481212,21.296875,4.699219,46548.000000,3,"German, Turkish, Arabic, English",3200,1220,2200,2023-06-29
39,"Lyon, France",10912,520774,365824,27.312500,6.898438,46226.000000,11,French,2300,1200,1800,2023-03-31


So we will correct it by replacing the formatting errors such as ';', '.' and double spaces with the same ', '.

In [464]:
corrected = []
for city in temp.City:
    if ';' in city:
        corrected.append(city.replace(';', ', '))
    elif '.'in city:
        corrected.append(city.replace('.', ','))
    elif '  ' in city:
        corrected.append(city.replace('  ', ' '))
    else:
        corrected.append(city)

temp.City = corrected

print(temp.iloc[2,0])
print(temp.iloc[12,0])
print(temp.iloc[15,0])
print(temp.iloc[36,0])

Brussels, Belgium
Lemesos, Cyprus
Berlin, Germany
Lyon, France


Another problem is that in row of index 46 the pair is in 'country, city' format instead of 'city, country', we can solve it by manually correcting it.

In [465]:
temp.loc[46]

City                                    Greece, Athens
Population Density                                1829
Population                                     3530371
Working Age Population                         2287174
Youth Dependency Ratio                            22.0
Unemployment Rate                            17.203125
GDP per Capita                                 38580.0
Days of very strong heat stress                     17
Main Spoken Languages                   Greek, English
Average Monthly Salary                            1050
Average Rent Price                                 600
Average Cost of Living                            1200
Last Data Update                   2024-07-16 00:00:00
Name: 46, dtype: object

In [466]:
temp.loc[46, 'City'] = 'Athens, Greece'

temp.loc[46]

City                                    Athens, Greece
Population Density                                1829
Population                                     3530371
Working Age Population                         2287174
Youth Dependency Ratio                            22.0
Unemployment Rate                            17.203125
GDP per Capita                                 38580.0
Days of very strong heat stress                     17
Main Spoken Languages                   Greek, English
Average Monthly Salary                            1050
Average Rent Price                                 600
Average Cost of Living                            1200
Last Data Update                   2024-07-16 00:00:00
Name: 46, dtype: object

Now that all pairs have consistent formatting we can split the pairs in CityName and Country.

In [467]:
#Extract the country

pairs = [city.split(', ') for city in temp.City]

cities = [pair[0] for pair in pairs]
countries = [pair[1] for pair in pairs]

print(cities)
print(countries)

['Vienna', 'Salzburg', 'Brussels', 'Antwerp', 'Gent', 'Bruges', 'Sofia', 'Dobrich', 'Zurich', 'Geneva', 'Basel', 'Lefkosia', 'Lemesos', 'Prague', 'Ostrava', 'Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt am Main', 'Stuttgart', 'Leipzig', 'Dresden', 'Dusseldorf', 'Hanover', 'Copenhagen', 'Odense', 'Madrid', 'Barcelona', 'Valencia', 'Seville', 'Malaga', 'Tallinn', 'Helsinki', 'Tampere', 'Paris', 'Lyon', 'Toulouse', 'London', 'Leeds', 'Glasgow', 'Liverpool', 'Edinburgh', 'Athens', 'Thessaloniki', 'Zagreb', 'Split', 'Budapest', 'Miskolc', 'Debrecen', 'Dublin', 'Cork', 'Rome', 'Milan', 'Naples', 'Turin', 'Florence', 'Venice', 'Luxembourg', 'Riga', 'Malta', 'The Hague', 'Amsterdam', 'Rotterdam', 'Utrecht', 'Eindhoven', 'Oslo', 'Bergen', 'Stavanger', 'Warsaw', 'Lodz', 'Cracow', 'Lisbon', 'Porto', 'Braga', 'Coimbra', 'Giroc', 'Bratislava', 'Ljubljana', 'Stockholm', 'Gothenburg', 'Malmo', 'Ankara', 'Adana']
['Austria', 'Austria', 'Belgium', 'Belgium', 'Belgium', 'Belgium', 'Bulgaria', 'Bu

Now that we have the cities and country lists we can add them to the dataframe.

In [468]:
temp['CityName'] = cities
temp['Country'] = countries

temp.loc[:,['City', 'CityName', 'Country']]

,City,CityName,Country
1,"Vienna, Austria",Vienna,Austria
2,"Salzburg, Austria",Salzburg,Austria
3,"Brussels, Belgium",Brussels,Belgium
4,"Antwerp, Belgium",Antwerp,Belgium
5,"Gent, Belgium",Gent,Belgium
...,...,...,...
82,"Stockholm, Sweden",Stockholm,Sweden
83,"Gothenburg, Sweden",Gothenburg,Sweden
84,"Malmo, Sweden",Malmo,Sweden
85,"Ankara, Turkiye",Ankara,Turkiye


Now, let's confirm that this process didn't create new duplicates.

In [469]:
temp[temp.duplicated(keep=False)]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update,CityName,Country


We just need to check if there is a duplicated city name in the dataset with different data associated with it. As we can check in the following output this doesn't happen in our dataset

In [470]:
temp[temp.duplicated(subset=['CityName'])]

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update,CityName,Country


Now we can make the changes to our DataFrame permanent by applying them to the original DataFrame.

In [471]:
df = temp

Finally we can count the name of rows (Cities) that have the same value in the 'Country' collumn.

In [472]:
df['Country'].value_counts()

Country
Germany            10
Italy               6
United Kingdom      5
Spain               5
Netherlands         5
Belgium             4
Portugal            4
Hungary             3
Poland              3
Sweden              3
France              3
Norway              3
Switzerland         3
Czechia             2
Turkiye             2
Greece              2
Denmark             2
Finland             2
Austria             2
Bulgaria            2
Cyprus              2
Croatia             2
Ireland             2
Estonia             1
Luxembourg          1
Latvia              1
Malta               1
Romania             1
Slovenia            1
Slovak Republic     1
Name: count, dtype: int64

This way we can conclude that the most frequent country is Germany and it has 10 cities associated with it.

#####        ```b) How many cities are present in total? How many are associated with Greece?```

As duplicate entries were solved in the previous question, the number of cities in the dataset is the number of rows in our dataset.

In [473]:
df.shape

(84, 15)

As we can see at there are 84 rows in our dataset, so 84 cities in total.

As for the number of cities associated with Greece, there are 2 cities associated with Greece (as per the next output)

In [474]:
print(df[df['Country'] == 'Greece'].shape[0], 'rows')
df.query('Country == "Greece"')

2 rows


,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update,CityName,Country
46,"Athens, Greece",1829,3530371,2287174,22.000000,17.203125,38580.0,17,"Greek, English",1050,600,1200,2024-07-16,Athens,Greece
47,"Thessaloniki, Greece",381,1050568,684564,22.203125,19.906250,23940.0,10,Greek,1000,550,1100,2023-12-21,Thessaloniki,Greece


#####        ``c) Which is the least spoken language in the dataset? Which are the top 3 most spoken languages?``

As we can see the `Most Spoken Language` column is formatted as a string of comma separated languages, so we will firstly need to convert it to a list of the languages spoken in that city.

In [475]:
df[['City', 'Main Spoken Languages']].head()

,City,Main Spoken Languages
1,"Vienna, Austria","German, English, Turkish, Serbian"
2,"Salzburg, Austria",German
3,"Brussels, Belgium","French, Dutch, Arabic, English"
4,"Antwerp, Belgium","Dutch, French, Arabic"
5,"Gent, Belgium","Dutch, French"


In [476]:
temp = df.copy()

In [477]:
temp['Main Spoken Languages'] = temp['Main Spoken Languages'].astype(str).str.split(', ')

temp.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update,CityName,Country
1,"Vienna, Austria",310,2983513,2018818,20.093750,10.203125,55770.0,3,"[German, English, Turkish, Serbian]",2500,1050,2061,2024-06-15,Vienna,Austria
2,"Salzburg, Austria",243,375489,250472,20.296875,3.000000,66689.0,0,[German],3200,1100,2186,2023-11-03,Salzburg,Austria
3,"Brussels, Belgium",681,3284548,2137425,27.500000,10.703125,62500.0,3,"[French, Dutch, Arabic, English]",3350,1200,1900,2023-04-22,Brussels,Belgium
4,"Antwerp, Belgium",928,1139663,723396,27.703125,6.199219,57595.0,3,"[Dutch, French, Arabic]",2609,900,1953,2024-08-09,Antwerp,Belgium
5,"Gent, Belgium",552,645813,417832,24.796875,4.585938,53311.0,2,"[Dutch, French]",2400,827,1200,2023-07-17,Gent,Belgium


Now that we have the languages in a list, we need to explode this collumn creating a new row for each language spoken in a city.

In [478]:
exploded_languages = temp.explode('Main Spoken Languages')

exploded_languages.head()

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Last Data Update,CityName,Country
1,"Vienna, Austria",310,2983513,2018818,20.093750,10.203125,55770.0,3,German,2500,1050,2061,2024-06-15,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.093750,10.203125,55770.0,3,English,2500,1050,2061,2024-06-15,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.093750,10.203125,55770.0,3,Turkish,2500,1050,2061,2024-06-15,Vienna,Austria
1,"Vienna, Austria",310,2983513,2018818,20.093750,10.203125,55770.0,3,Serbian,2500,1050,2061,2024-06-15,Vienna,Austria
2,"Salzburg, Austria",243,375489,250472,20.296875,3.000000,66689.0,0,German,3200,1100,2186,2023-11-03,Salzburg,Austria


Finally, we can check how many times each language apears in the `exploded_languages` dataframe.

In [479]:
exploded_languages['Main Spoken Languages'].value_counts()

Main Spoken Languages
English              47
German               15
Turkish              10
French               10
Dutch                 9
Italian               6
Swedish               4
Greek                 4
Polish                4
Portuguese            4
Spanish               4
Hungarian             3
Russian               3
Arabic                3
Norwegian             3
Urdu                  2
Danish                2
Czech                 2
Finnish               2
Bulgarian             2
Irish Gaelic          2
Croatian              2
Serbian               1
Estonian              1
  Arabic              1
Catalan               1
Spanish;Valencian     1
Scots Gaelic          1
Scots                 1
Bengali               1
Latvian               1
Luxembourgish         1
Maltese               1
Romanian              1
Slovak                1
Slovene               1
Name: count, dtype: int64

The least spoken language is Slovene and the 3 most spoken are English, German and Turkish.

#### 3. 

        


##### ``a) Entries uploaded before April 2023 need to be updated. Which cities would require an update?``

To check for entries uploaded before April 2023, we filtered our dataset using the ``datetime`` library.

In [480]:
need_update_df = df[df['Last Data Update'] < dt.datetime(year = 2023, month = 4, day = 1)]
cities = need_update_df['CityName'].to_list()
cities

['Basel', 'Munich', 'Frankfurt am Main', 'Lyon', 'Venice', 'Gothenburg']

``Basel``, ``Munich``, ``Frankfurt am Main``, ``Lyon``, ``Venice`` and ``Gothenburg`` would require an update.

#####        ``b) How many days ago was the last update? On what day, month, and year did it occur?``


In [481]:
now = dt.datetime.now() #to calculate the distance

last_update = max(df['Last Data Update'])

#Show results
print(f'The last update was {(now - last_update).days} days ago, on the {last_update.day}th of {last_update.month_name()} {last_update.year}.')

The last update was 374 days ago, on the 12th of December 2024.


#### 4. 



##### ``a) How are the Unemployment Rate and GDP per Capita distributed and related? What does this relationship suggest? Provide a visual representation.``

First we need to import the necessary graph libraries.

In [482]:
# Import libraries and define the alias
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import plotly.express as px
import plotly.graph_objects as go

Then we need to draw the graph (comments on the code).

In [483]:
# Convert 'GDP per Capita' and 'Unemployment Rate' to float32 for more accurate numerical computations
x = df['GDP per Capita'].astype('float32') 
y = df['Unemployment Rate'].astype('float32')

# linear regression
slope, intercept = np.polyfit(x, y, 1) # Fit the linear regression (fit a line y = slope*x + intercept)
y_fit = slope * x + intercept          # Calculate the predicted y values from the regression line

fig = go.Figure()                      # Create a new plotly figure

# Add a scatter plot of the original data points
fig.add_trace(go.Scatter(x = x, y = y, 
                         mode='markers', marker= dict(color="#914678", size=10), 
                         name = 'Relation'))

# Add the regression trendline to the plot
fig.add_trace(go.Scatter(x=x, y=y_fit, 
                         mode='lines', line= dict(color = "#58F011", width=2), 
                         name='Trendline'))

# Update layout: title formatting and positioning
fig.update_layout(title = dict(text="<b>GDP per Capita and Unemployment Rate Distribution</b>",  
                                font= dict(family='Arial', 
                                         size=20, color='#000000'),
                                xanchor = 'center', x = 0.5, yanchor='top', y=0.9))

# Set x-axis title and font
fig.update_xaxes(title_text='GDP per Capita', 
                 title_font=dict(family='Arial', 
                                 size=12, color='#000000'))

# Set y-axis title and font
fig.update_yaxes(title_text='Unemployment Rate', 
                 title_font=dict(family='Arial', 
                                 size=12, color='#000000'))

# Display the figure
fig.show()

The relation between the Unemployent Rate and the GDP per Capita suggests that in cities with a higher Unemployment Rate the GDP per Capita is lower.

#####        ``b) Which are the top 5 cities with the largest difference between the Average Monthly Salary and Average Cost of Living? What about the top 5 countries with the smallest average difference?``
Show these results with meaningful visualizations.


To avoid modifying the original dataframe, we first created a copy.

In [484]:
city_data_copy = df.copy()

We calculate the **Average Savings** for each city as the difference between the ``Average Monthly Salary`` and the ``Average Cost of Living``. 


In [485]:
city_data_copy['Average Savings'] = city_data_copy['Average Monthly Salary'] - city_data_copy['Average Cost of Living']

In [486]:
# Sort cities by 'Average Savings' to help identify the largest 'Average Savings'
city_data_sorted = city_data_copy.sort_values(by='Average Savings')

fig = go.Figure() # Create a new plotly figure

# Add a bar chart for each city showing the Average Savings
fig.add_trace(go.Bar(x = city_data_sorted['CityName'], y = city_data_sorted['Average Savings'], 
                     marker=dict(color="#F011A6"),))

# Set axis titles and figure size
fig.update_layout(xaxis_title = 'City', 
                  yaxis_title = 'Difference', 
                  width=1300, height=500)

# Add the title
fig.update_layout(title = dict(text="<b>Difference between Average Monthly Salary and Average Cost of Living by City</b>",  
                               font=dict(family='Arial', 
                                         size=17, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.9),
                  font = dict(size = 10))

fig.show()

The top 5 cities with the largest difference between the Average Monthly Salary and Average Cost of Living are (in descending order):
Basel, Tampere, Helsinki, Zurich and Geneve.

-----------------------------------------------------------

To anwser the second part of the question, we first need to group the dataframe by the average ``Average Savings`` for each ``Country``

In [487]:
savings_per_country = pd.pivot_table(city_data_copy, index='Country', 
                                    values='Average Savings', 
                                    aggfunc={'Average Savings':'mean'}).round(2).reset_index()
savings_per_country.head()

,Country,Average Savings
0,Austria,726.5
1,Belgium,1074.5
2,Bulgaria,150.0
3,Croatia,245.0
4,Cyprus,168.0


In [488]:
# Sort countries by 'Average Savings' to help identify the smallest values
savings_per_country_sorted = savings_per_country.sort_values(by='Average Savings')


fig = go.Figure() # Create a new plotly figure

# Add a bar chart for each country showing the Average Savings
fig.add_trace(go.Bar(x = savings_per_country_sorted['Country'], y = savings_per_country_sorted['Average Savings'], 
                     marker=dict(color="#A2F011")))

# Add the axis names and set figure size
fig.update_layout(xaxis_title = 'Country', yaxis_title = 'Difference', 
                  width=900, height=500)

# Set figure title 
fig.update_layout(title = dict(text="<b>Difference between Average Monthly Salary and Average Cost of Living by Country</b>",  
                               font=dict(family='Arial', size=17, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.9))

fig.show() # Show f

The top 5 countries with the smallest average difference are (in ascending order):
Portugal, Greece, Slovak Republic, Turkiye and Poland.

##### ``c) Which is the best city for someone seeking:``

- an average monthly salary above €2000
- a cost of living below €1600
- the lowest unemployment rate possible



First we need to filter the dataframe to only include cities with an ``Average Monthly Salary`` above 2000€.

In [489]:
city_data_filtered = city_data_copy[(city_data_copy['Average Monthly Salary'] > 2000)]

Then we do the same for ``Average Cost of Living`` bellow 1600€.

In [490]:
city_data_filtered = city_data_filtered[(city_data_filtered['Average Cost of Living'] < 1600)]

Finally, we sort the filtered dataframe by ``Unemployment Rate``, such that the city with the lowest value is on the first row.

In [491]:
city_data_filtered_sorted = city_data_filtered.sort_values(by = 'Unemployment Rate')
city_data_filtered_sorted[['City', 'Average Monthly Salary', 'Average Cost of Living', 'Unemployment Rate']]

,City,Average Monthly Salary,Average Cost of Living,Unemployment Rate
24,"Dresden, Germany",3050,1530,3.800781
5,"Gent, Belgium",2400,1200,4.585938
29,"Madrid, Spain",2271,1589,10.796875
31,"Valencia, Spain",2400,1300,13.703125
33,"Malaga, Spain",2200,1400,17.703125
32,"Seville, Spain",2300,1150,21.593750


The best city for someone seeking those specific conditions is Dresden, Germany.

#### 5. What are three additional insights you find meaningful when comparing the given cities?

While choosing a city to live, it is important to understand how our finacial balance looks at the end of the month. For that, we built a graph that shows the percentage of money that, after all the payments associated with living in that city were made, stays in the person's pocket, in order of *Average Monthly Salary* by city in ascending order.

In [492]:
retention_tax = df.copy()
retention_tax["Retention Rate of Salary"] = round((retention_tax['Average Monthly Salary'] - retention_tax['Average Cost of Living'])/retention_tax['Average Monthly Salary']*100, 2)

In [493]:
retention_tax = retention_tax.sort_values('Retention Rate of Salary')
fig = go.Figure()
fig.add_trace(go.Bar(x = retention_tax['CityName'], y = retention_tax['Retention Rate of Salary'], 
                     marker=dict(color="#B511F0")))
fig.update_layout(xaxis_title = 'City', yaxis_title = 'Percentage (%)', width=1300, height=500)
fig.update_layout(title = dict(text="<b>Average Saving Rate of Salary</b>",  
                               font=dict(family='Arial', size=17, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.9))
fig.update_layout(font = dict(size = 10))
fig.show()

The cities with higher ``Average Saving Rate of Salary`` are **Tampere**, **Seville** and **Gent**, and the ones with lower are **Lisbon**, **Porto** and **Athens**.

--------------------------------------

If we are planning to move to a city and we are in job market, it is important to know the umployment rate. Below, we show the cities where this rate is above 10%.

In [494]:
data = df.sort_values('Unemployment Rate', ascending = True)
fig = go.Figure()
fig.add_trace(go.Bar(x = data['CityName'], y = data['Unemployment Rate'], 
                     marker=dict(color="#F0B811"),
                     hovertemplate=
                    "<b>City:</b> %{x}<br>" +
                    "<b>Unemployment Rate:</b> %{y:.1f}%<br>" +
                    "<extra></extra>"))
fig.update_layout(xaxis_title = 'City', yaxis_title = 'Percentage (%)', width=1300, height=500)
fig.update_layout(title = dict(text="<b>Unemployment Rate</b>",  
                               font=dict(family='Arial', size=17, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.9))
fig.update_layout(font = dict(size = 10))

fig.show()

The cities with higher ``Unemployment Rate`` are **Naples**, **Seville** and **Thessalonikl**, and the ones with lower are **Prague**, **Stavanger** and **Munich**.

-----------

Due to medical conditions, the number of *Days of very strong heat stress* might be a crucial factor. Below, we show all the cities where that number is bigger than 10 and then where is smaller than 3.

In [495]:
data = df.sort_values('Days of very strong heat stress', ascending = True)
fig = go.Figure()
fig.add_trace(go.Bar(x = data['CityName'], y = data['Days of very strong heat stress'], 
                     marker=dict(color="#1175F0"),
                     hovertemplate=
                    "<b>City:</b> %{x}<br>" +
                    "<b>Days of very strong heat stress:</b> %{y}<br>" +
                    "<extra></extra>"))
fig.update_layout(xaxis_title = 'City', yaxis_title = 'Number of Days', width=1300, height=500)
fig.update_layout(title = dict(text="<b>Days of very strong heat stress</b>",  
                               font=dict(family='Arial', size=17, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.9))
fig.update_layout(font = dict(size = 10))
fig.show()

The cities with higher ``Days of Very Strong Heat Stress`` are **Adana**, **Seville** and **Lefkosia**, and there are several cities with 0 days of very strong heat stress.

## Advanced Topic - Building an Interactive Map

### 1. Web Scraping

To get the exact geographical coordinates for each city we need to webscrape for each city's wikipedia entry and from there get the latitude and longitude for each of them.

#### Importing Libraries needed for web scrapping

In [496]:
from bs4 import BeautifulSoup               # process html

from selenium import webdriver              # automate web browser interaction
from selenium.webdriver.common.by import By # specify how to locate elements on a web page

import time                                 # set time delays

#### Create a browser instance

In [497]:
from selenium.webdriver.chrome.options import Options

# The following lines of code were needed for the browser to not crash
options = Options()
options.add_argument("--headless=new")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-software-rasterizer")


browser = webdriver.Chrome(options=options) # creates a new instance of a Chrome browser

url = 'https://en.wikipedia.org/wiki/Main_Page'


#### Get the city's coordinates

In [498]:
from lat_lon_parser import parse    #This library converts coordinates from Degrees, Minutes, Seconds to Decimal Degrees

In [499]:
def get_coordinates(city):
    try:
        browser.get(url) # opens the main wikipedia page

        searchbox = browser.find_element(By.CLASS_NAME, 'cdx-text-input__input') # finds the wikepedia search bar
        searchbox.clear() # clears all previous inputs from the search bar
        searchbox.send_keys(city) # inserts the city + country into the searchbar
        time.sleep(2)

        firstresult = browser.find_element(By.CLASS_NAME, 'cdx-menu-item__content') # finds the first suggested result
        firstresult.click() # clicks on the first suggestion
        time.sleep(2)
    
        #If the link leads to a search result page instead of a city page, we neet the first result
        page_title = browser.title
        if "Search results" in page_title:
            results = browser.find_elements(By.CSS_SELECTOR, ".mw-search-result-heading a") # finds the first result
            if results:
                results[0].click() # clicks on it
                time.sleep(2)

        if 'Nan' in page_title:
            return None, None
        
        html_source = browser.page_source # gets the page source code
        print(browser.title)
        
        soupObj = BeautifulSoup(html_source, 'html.parser') # parses the source code into a soup object

        lat = soupObj.find(attrs={"class": 'latitude'}).text   # gets the city latitude
        lon = soupObj.find(attrs={"class": 'longitude'}).text # gets the city latitude

        return lat, lon
    
    except:
        return None, None

The following code cell calls the function defined above, searching the wikipedia site first by the City Name and if the result page does not return a page with coordinates, searches for the City + Country pair from ``City``. The cell is commented because it takes too much time to run. So, this cell saves the dataframe containing each cities coordinates as a csv file that is read after.


In [500]:
'''
latitudes = []
longitudes = []

for i in range(len(df)):
    searchfor = df['CityName'].iloc[i]

    lat, lon = get_coordinates(searchfor)

    if lat is None:
        # fallback search
        searchfor = df['City'].iloc[i]
        lat, lon = get_coordinates(searchfor)

    # if still None skip
    if lat is None:
        continue

    # convert and store
    latitudes.append(parse(lat))
    longitudes.append(parse(lon))

    
# Add the latitude and longitude lists as collumns to the end of the data frame
df['Latitude'] = latitudes
df['Longitude'] = longitudes

df.to_csv('ct_dataset.csv')

df.head()
'''

"\nlatitudes = []\nlongitudes = []\n\nfor i in range(len(df)):\n    searchfor = df['CityName'].iloc[i]\n\n    lat, lon = get_coordinates(searchfor)\n\n    if lat is None:\n        # fallback search\n        searchfor = df['City'].iloc[i]\n        lat, lon = get_coordinates(searchfor)\n\n    # if still None skip\n    if lat is None:\n        continue\n\n    # convert and store\n    latitudes.append(parse(lat))\n    longitudes.append(parse(lon))\n\n\n# Add the latitude and longitude lists as collumns to the end of the data frame\ndf['Latitude'] = latitudes\ndf['Longitude'] = longitudes\n\ndf.to_csv('ct_dataset.csv')\n\ndf.head()\n"

#### Closing browser instance

In [501]:
browser.quit() #close the browser instance.

### 2. Interactive Map

The interactive map was created using the **Plotly Graph Objects** library to visualize the geographic distribution of cities from the dataset.

First, we extratcted of the latitude and longitude columns from the csv file (`ct_dataset.csv`) using pandas.

In [502]:
df = pd.read_csv('ct_dataset.csv')
latitudes = df['Latitude']
longitudes = df['Longitude']

Then, we created a **scattermap** where each marker corresponds to a city in the dataset.

- `lat` and `lon` define the geographic coordinates of each point;
- `mode= 'markers'` specifies that only point markers are displayed;
- `marker` controls the size and color of each city marker;
- `hovertext` displays the name of the city;
- `customdata` stores additional city attributes used in the toolti; 
- `hovertemplate` formats the information shown when hovering over a city;
- `hoverlabel` customizes the appearance of the tooltip.


Customization of the map's layout:

- The map is centered over Europe (latitude 50, longitude 10);  
- Hover mode is set to `closest`;
- Addition of a descriptive title;
- Definition of the figure size.  


In [503]:
fig = go.Figure()
fig.add_trace(go.Scattermap(lat = latitudes, lon = longitudes, 
                            mode = 'markers', 
                            marker = dict(size=10, color = "#538563"),
                            hovertext = df['CityName'],
                            hoverinfo = 'text',
                            customdata = df[['Country','Population','Average Monthly Salary','Average Cost of Living']].values,
                            hovertemplate = "<b>%{hovertext}</b><br>" +
                                            "Country: %{customdata[0]}<br>" +
                                            "Population: %{customdata[1]}<br>" +
                                            "Average Monthly Salary: %{customdata[2]}<br>" +
                                            "Average Cost of Living: %{customdata[3]}<br>",
                            hoverlabel=dict(bgcolor="#538563",
                                            font_size=14,
                                            font_family="Arial",
                                            font_color="#FFFFFF",
                                            bordercolor="#FFFFFF"),
                            name = ''))


fig.update_layout(hovermode = 'closest', 
                    map = dict(bearing = 0, 
                                center=go.layout.map.Center(lat=50, lon=10), 
                                pitch = 0, zoom = 3),
                    title = dict(text='<b>Where Should I Live?<br><b><sup>(hover for city details)</sup>',  
                               font=dict(family='Arial', size=23, color='#000000'),
                               xanchor = 'center', x = 0.5, yanchor='top', y=0.95),
                    width=1200, height=800)

The final result is a fully interactive geographic visualization where users can zoom and hover over cities to explore detailed socioeconomic information.

## Data Science In Action

For the Data Science In Action we decided to create a recommendation system integrated in a Graphic User Interface. This allows the user to choose a language and condition parameters of the users choice. In the end the user gets a **Recommended City** which corresponds to the city that matches the users preferences the most and a table with the **Top Matching Cities** which corresponds to the 5 cities that match the users preferences the most.

### Importation and initialization of Panel.

- `pn.extension(design="material")` activates Panel and applies the Material theme.

In [504]:
import panel as pn

pn.extension(design="material")

### Title

In [505]:
title = pn.pane.Markdown("# Where Should I Live?")

### Sidebar Widgets

Creation of multiple interactive widgets to collect user preferences:
  
- A **dropdown selector** allows users to choose their preferred language.  
- Multiple **sliders** define numeric constraints for:
  - `pop_density`
  - `youth_dependency`
  - `unemployment_rate`
  - `gdp` 
  - `heat_days`
  - `salary`
  - `rent`
  - `living_cost` 
- Each slider is paired with a **checkbox** that allows users to include or exclude that variable from the filtering process.  
- A **button** triggers the city recommendation process.

    pop_density
    youth_dependency
    unemployment_rate
    gdp 
heat_days
salary
rent
living_cost


In [506]:
language_select = pn.widgets.Select(
    name="Select Preferred Language",
    options=exploded_languages['Main Spoken Languages'].unique().tolist()
)

pop_density_slider = pn.widgets.IntSlider(
    name="Population Density",
    start=200,
    end=21000,
    value=500,
    step=10)
use_pop_density = pn.widgets.Checkbox(name="Use Population Density", value=True)

youth_dependency_slider = pn.widgets.IntSlider(
    name="Youth Dependency Ratio",
    start=20,
    end=40,
    value=30,
    step=1)
use_youth_dependency = pn.widgets.Checkbox(name="Use Youth Dependency", value=True)

unemployment_slider = pn.widgets.IntSlider(
    name="Unemployment Rate",
    start=3,
    end=23,
    value=10,
    step=1)
use_unemployment = pn.widgets.Checkbox(name="Use Unemployment", value=True)

gdp_slider = pn.widgets.IntSlider(
    name="GDP per Capita",
    start=5000,
    end=80000,
    value=20000,
    step=1000)
use_gdp = pn.widgets.Checkbox(name="Use GDP per Capita", value=True)

heat_slider = pn.widgets.IntSlider(
    name="Days of Very Strong Heat Stress",
    start=0,
    end=120,
    value=30,
    step=5)
use_heat = pn.widgets.Checkbox(name="Use Heat Stress", value=True)

salary_slider = pn.widgets.IntSlider(
    name="Average Monthly Salary",
    start=500,
    end=6000,
    value=2000,
    step=100)
use_salary = pn.widgets.Checkbox(name="Use Salary", value=True)

rent_slider = pn.widgets.IntSlider(
    name="Average Rent Price",
    start=300,
    end=4000,
    value=1200,
    step=100)
use_rent = pn.widgets.Checkbox(name="Use Rent", value=True)

living_cost_slider = pn.widgets.IntSlider(
    name="Average Cost of Living",
    start=600,
    end=5000,
    value=1800,
    step=100)
use_living_cost = pn.widgets.Checkbox(name="Use Living Cost", value=True)

### Find button

In [507]:
find_button = pn.widgets.Button(name="Find My City", button_type="primary")

### Output Areas

Creation of several dynamic output components to display results:

- `result_text` displays the recommended city and its key indicators.  
- `metrics_row` is reserved for additional metric visualizations.  
- `table_output` shows a dataframe with the top matching cities.

In [508]:
result_text = pn.pane.Markdown("")
metrics_row = pn.Row()
table_output = pn.pane.DataFrame(width=900)

### Callback Function

The `find_city` function is executed when the button is clicked and performs the full recommendation logic:

- **Language filtering** is applied first using the selected language.  
- **Numeric filters** are conditionally applied only if their corresponding checkboxes are active.  
- A **scoring system** computes the distance between each city’s attributes and the user’s selected values and divides it by the selected value (if we want to maximize the attribute (eg. GDP) the distance is subtracted from the score, otherwise we add it to the score);  
- The city with the **lowest score** is selected as the best match.  
- If no city satisfies the conditions, a message is displayed to the user.  
- The **top 5 best-matching cities** are displayed in a table.

In [509]:
def find_city(event):
    language = language_select
    pop_density = pop_density_slider.value
    youth_dependency = youth_dependency_slider.value
    unemployment_rate = unemployment_slider.value
    gdp = gdp_slider.value
    heat_days = heat_slider.value
    salary = salary_slider.value
    rent = rent_slider.value
    living_cost = living_cost_slider.value

    
    # FILTER BY LANGUAGE
    language = language_select.value
    filtered = df[df["Main Spoken Languages"].astype(str).str.contains(language, na=False)]


    # NUMERIC FILTERS 
    conditions = []

    if use_pop_density.value:
        conditions.append(filtered["Population Density"] <= pop_density)

    if use_youth_dependency.value:
        conditions.append(filtered["Youth Dependency Ratio"] <= youth_dependency)

    if use_unemployment.value:
        conditions.append(filtered["Unemployment Rate"] <= unemployment_rate)

    if use_gdp.value:
        conditions.append(filtered["GDP per Capita"] >= gdp)

    if use_heat.value:
        conditions.append(filtered["Days of very strong heat stress"] <= heat_days)

    if use_salary.value:
        conditions.append(filtered["Average Monthly Salary"] >= salary)

    if use_rent.value:
        conditions.append(filtered["Average Rent Price"] <= rent)

    if use_living_cost.value:
        conditions.append(filtered["Average Cost of Living"] <= living_cost)

    if conditions:
        filtered = filtered.loc[np.logical_and.reduce(conditions)]


    # SCORING SYSTEM
    score = 0

    if use_pop_density.value:
        score += abs(filtered["Population Density"] - pop_density)/pop_density

    if use_youth_dependency.value:
        score += abs(filtered["Youth Dependency Ratio"] - youth_dependency)/youth_dependency

    if use_unemployment.value:
        score += abs(filtered["Unemployment Rate"] - unemployment_rate)/unemployment_rate

    if use_gdp.value:
        score -= abs(filtered["GDP per Capita"] - gdp)/gdp

    if use_heat.value:
        score += abs(filtered["Days of very strong heat stress"] - heat_days)/(heat_days+1e-6) # A very small value was added to prevent dividing by 0 errors

    if use_salary.value:
        score -= abs(filtered["Average Monthly Salary"] - salary)/salary

    if use_rent.value:
        score += abs(filtered["Average Rent Price"] - rent)/rent

    if use_living_cost.value:
        score += abs(filtered["Average Cost of Living"] - living_cost)/living_cost

    filtered["Score"] = score

    if filtered.empty:
        result_text.object = (
        "## Recommended City\n\n"
        "No city matches your current preferences.\n\n"
        "Try adjusting your filters.")
        metrics_row.clear()
        table_output.object = None
        return

    best_city = filtered.sort_values(by="Score").iloc[0]
    result_text.object = (
    "# Recommended City\n\n"
    f"## **{best_city['CityName']}**, {best_city['Country']}\n\n"
    f"- Population Density: {best_city['Population Density']}\n"
    f"- Youth Dependency Ratio: {best_city['Youth Dependency Ratio']}\n"
    f"- Unemployment Rate: {best_city['Unemployment Rate']}%\n"
    f"- GDP per Capita: {best_city['GDP per Capita']}\n"
    f"- Average Monthly Salary: {best_city['Average Monthly Salary']}\n"
    f"- Average Rent Price: {best_city['Average Rent Price']}\n"
    f"- Average Cost of Living: {best_city['Average Cost of Living']}\n"
    f"- Days of Very Strong Heat Stress: {best_city['Days of very strong heat stress']}\n\n"
    f"**Match Score:** {best_city['Score']:.2f}")


    # TOP 5 MATCHES TABLE
    table_output.object = filtered.sort_values(by="Score").head(5)[
        ["CityName", "Country", "Population Density",
        "Youth Dependency Ratio", "Unemployment Rate",
        "GDP per Capita", "Days of very strong heat stress",
        "Average Monthly Salary", "Average Rent Price","Average Cost of Living"]]


### Button Binding

The recommendation function is connected to the button using:

- `find_button.on_click(find_city)` to trigger the search when the user clicks the button.

In [510]:
find_button.on_click(find_city)

Watcher(inst=Button(button_type='primary', design=<class 'panel.theme.materi..., name='Find My City'), cls=<class 'panel.widgets.button.Button'>, fn=<function find_city at 0x7211b62191c0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

### Sidebar Layout

The dashboard layout is organized into two main sections:

- A **sidebar** containing all filters and controls.  
- A **main panel** displaying:
  - The title  
  - The recommended city  
  - The top matching cities table  

These components are combined using `pn.Row` and `pn.Column` to form the final graphical user interface.

In [511]:
sidebar = pn.Column(
    "## Conditions",
    language_select,
    use_pop_density, pop_density_slider,
    use_youth_dependency, youth_dependency_slider,
    use_unemployment, unemployment_slider,
    use_gdp, gdp_slider,
    use_heat, heat_slider,
    use_salary, salary_slider,
    use_rent, rent_slider,
    use_living_cost, living_cost_slider,
    find_button,
    width=320)

### Main App Layout

The final interactive dashboard allows users to dynamically adjust their preferences and instantly receive personalized city recommendations based on multiple socioeconomic and environmental indicators.

In [512]:
gui = pn.Row(
    sidebar,
    pn.Column(
        title,
        result_text,
        metrics_row,
        pn.pane.Markdown("### Top Matching Cities"),
        table_output))

In [513]:
gui

BokehModel(combine_events=True, render_bundle={'docs_json': {'983979f0-a5f2-4bb1-934f-716bf7f67a2d': {'version…